In [1]:
#Optional: upload the train/test data to Google Colab. if you use it.
from google.colab import drive
drive.mount('/content/drive')
!unzip -q "/content/drive/My Drive/input_data.zip" -d /content

Mounted at /content/drive


In [2]:
!pip install pytorch_lightning torchmetrics wandb

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 777.7/777.7 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 806.1/806.1 kB 39.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 kB 23.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.1/254.1 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 10.6 MB/s eta 0:00:00


In [3]:
import pytorch_lightning as pl
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, random_split, Subset
from sklearn.model_selection import train_test_split
import torch
from PIL import Image
import numpy as np
import pandas as pd
import tifffile as tiff
from torchmetrics import F1Score
import os
import csv
from tqdm.notebook import tqdm
import random
import matplotlib.pyplot as plt
from types import SimpleNamespace

import wandb
from pytorch_lightning.loggers import WandbLogger

os.environ["WANDB_MODE"] = "online"
from google.colab import userdata
userdata.get('wandb_key') #Remember to input your W&B key onto the Secrets if you use it.

debug = False

config = {
    "batch_size" : 16,
    "num_workers": os.cpu_count(),
    "img_size" : 384,
    "max_epochs" : 3 if debug else 20,
}

config = SimpleNamespace(**config)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [4]:
def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
seed_everything(seed=42)

In [5]:
def load_and_convert_tiff(file_path):
    image = tiff.imread(file_path)
    R = image[:, :, 3]*255*2
    G = image[:, :, 2]*255*2
    B = image[:, :, 1]*255*2
    rgb_image = np.stack((R, G, B), axis=2).astype(np.uint8)
    rgb_image = Image.fromarray(rgb_image)
    return rgb_image

In [6]:
if debug:
    pos_tiff_files = [
        'input_data/train/train/train_3.tif',
        'input_data/train/train/train_12.tif',
        'input_data/train/train/train_14.tif',
        'input_data/train/train/train_15.tif',
    ]
    plt.figure(figsize=(8, 8))
    plt.suptitle('Positives')
    for i, file_path in enumerate(pos_tiff_files):
        rgb_image = load_and_convert_tiff(file_path)
        plt.subplot(2, 2, i+1)
        plt.imshow(rgb_image)
        plt.axis('off')
    plt.show()

    neg_tiff_files = [
        'input_data/train/train/train_0.tif',
        'input_data/train/train/train_1.tif',
        'input_data/train/train/train_2.tif',
        'input_data/train/train/train_4.tif',
    ]
    plt.figure(figsize=(8, 8))
    plt.suptitle('Negatives')
    for i, file_path in enumerate(neg_tiff_files):
        rgb_image = load_and_convert_tiff(file_path)
        plt.subplot(2, 2, i+1)
        plt.imshow(rgb_image)
        plt.axis('off')
    plt.show()

In [7]:
#Dataset class
class CustomImageDataset(Dataset):
    def __init__(self, csv_file, img_dir):
        self.img_labels = pd.read_csv(csv_file)
        self.img_dir = img_dir
        self.transform = transforms.Compose([
            transforms.Resize(config.img_size),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = load_and_convert_tiff(img_path)
        label = self.img_labels.iloc[idx, 1]
        image = self.transform(image)
        return image, label

def create_datasets(csv_file, img_dir, val_split=0.1):
    full_dataset = CustomImageDataset(csv_file=csv_file, img_dir=img_dir)
    labels = full_dataset.img_labels.iloc[:, 1].tolist()
    train_idx, val_idx = train_test_split(
        range(len(labels)),
        test_size=val_split,
        stratify=labels,
        random_state=42,
    )

    train_dataset = Subset(full_dataset, train_idx)
    val_dataset = Subset(full_dataset, val_idx)

    train_labels = [labels[i] for i in train_idx]
    val_labels = [labels[i] for i in val_idx]
    print("train_size:", len(train_dataset), "with pos:", train_labels.count(1))
    print("val_size:", len(val_dataset), "with neg:", val_labels.count(1))

    return train_dataset, val_dataset

train_dataset, val_dataset = create_datasets(csv_file='input_data/train/answer.csv', img_dir='input_data/train/train')

train_size: 1116 with pos: 230
val_size: 125 with neg: 26


In [8]:
# Model
class ImageClassifier(pl.LightningModule):
    def __init__(self):
        super(ImageClassifier, self).__init__()
        self.model = torch.hub.load('hankyul2/EfficientNetV2-pytorch', 'efficientnet_v2_s', pretrained=True, nclass=2)
        self.f1 = F1Score(num_classes=2, task='binary')

    def forward(self, x):
        return self.model(x)

    def training_step(self, batch, batch_idx):
        x, y = batch
        assert y.min() >= 0 and y.max() <= 1, f"Labels out of range: min={y.min()}, max={y.max()}"
        logits = self(x)

        loss = torch.nn.functional.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)

        f1_score = self.f1(preds, y)
        self.log('train_loss', loss)
        self.log('train_f1', f1_score, on_step=False, on_epoch=True, prog_bar=True)
        return loss

    def validation_step(self, batch, batch_idx):
        x, y = batch
        assert y.min() >= 0 and y.max() <= 1, f"Labels out of range: min={y.min()}, max={y.max()}"

        logits = self(x)
        loss = torch.nn.functional.cross_entropy(logits, y)
        preds = torch.argmax(logits, dim=1)

        f1_score = self.f1(preds, y)
        self.log('val_loss', loss)
        self.log('val_f1', f1_score, on_step=False, on_epoch=True, prog_bar=True)
        return loss

    def train_dataloader(self):
        dataloader = DataLoader(train_dataset, batch_size=config.batch_size, num_workers=config.num_workers, shuffle=True)
        return dataloader

    def val_dataloader(self):
        val_loader = DataLoader(val_dataset, batch_size=config.batch_size, num_workers=config.num_workers, shuffle=False)
        return val_loader

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=2e-5)
        #optimizer = torch.optim.AdamW(self.parameters(), lr=2e-5, betas=(0.9, 0.999), eps=1e-8, weight_decay=1e-3)
        return optimizer

In [9]:
model = ImageClassifier()
train_dataloader = model.train_dataloader()
if debug:
    trainer = pl.Trainer(max_epochs=config.max_epochs, log_every_n_steps=1)
    trainer.fit(model, train_dataloader)
else:
    wandb.init(project='solafune-mining-colab')
    wandb_logger = WandbLogger()
    trainer = pl.Trainer(max_epochs=config.max_epochs, logger=wandb_logger) #, log_every_n_steps=20
    trainer.fit(model, train_dataloader)
    run_name = wandb.run.name
    wandb.finish()
    torch.save(model.state_dict(), f'my_model_{run_name}.pth')

/usr/local/lib/python3.10/dist-packages/torch/hub.py:294: UserWarning: You are about to download and run code from an untrusted repository. In a future release, this won't be allowed. To add the repository to your trusted list, change the command to {calling_fn}(..., trust_repo=False) and a command prompt will appear asking for an explicit confirmation of trust, or load(..., trust_repo=True), which will assume that the prompt is to be answered with 'yes'. You can also use load(..., trust_repo='check') which will only prompt for confirmation if the repo is not already trusted. This will eventually be the default behaviour
  warnings.warn(
Downloading: "https://github.com/hankyul2/EfficientNetV2-pytorch/zipball/main" to /root/.cache/torch/hub/main.zip


<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
/usr/local/lib/python3.10/dist-packages/pytorch_lightning/loggers/wandb.py:389: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name  | Type           | Params
-----------------------------------------
0 | model | EfficientNetV2 | 20.2 M
1 | f1    | BinaryF1Score  | 0     
-----------------------------------------
20.2 M    Trainable params
0         Non-trainable params
20.2 M    Total params
80.

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_epochs=20` reached.


epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇████
train_f1,▁▃▄▅▆▇▆▇▇█▇▇████████
train_loss,█▇▆▆▄▃▆▃▃▂▂▁▁▂▂▁▃▁▁▂▁▁▂▂▁▃▁▁
trainer/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
val_f1,▁▃▅▅▆▇▄▇▆▇█▇▇▆▅▆▇█▆█
val_loss,█▇▅▄▃▂▂▂▁▁▂▂▂▁▂▁▁▁▁▁
epoch,19
train_f1,0.9834
train_loss,0.02799
trainer/global_step,1399
val_f1,0.87906


In [10]:
#Inference
class InferenceDataset(Dataset):
    def __init__(self, img_dir,file_names):
        self.img_dir = img_dir
        self.img_names = file_names
        self.transform = transforms.Compose([
            transforms.Resize(config.img_size),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

    def __len__(self):
        return len(self.img_names)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_names[idx])
        image = load_and_convert_tiff(img_path)
        image = self.transform(image)
        return image, self.img_names[idx]

file_names = []
with open('input_data/uploadsample.csv', newline='') as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        file_names.append(row[0])

inference_dataset = InferenceDataset(img_dir='input_data/evaluation_images', file_names=file_names)
inference_loader = DataLoader(inference_dataset, batch_size=config.batch_size, num_workers=config.num_workers, shuffle=False)

In [11]:
model.eval()
model.to(device)
with open(f'my_answer_{run_name}.csv', 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    predictions = []
    file_names = []
    with torch.no_grad():
        for data, file_paths in tqdm(inference_loader):
            data = data.to(device)
            logits = model(data)
            preds = torch.argmax(logits, dim=1)
            predictions.extend(preds.cpu().numpy())
            file_names.extend(file_paths)

    combined = list(zip(file_names, predictions))
    for file_name, prediction in combined:
        writer.writerow([file_name, prediction])

print(predictions.count(1), "positives")

  0%|          | 0/78 [00:00<?, ?it/s]

253 positives
